In [6]:
#!/usr/bin/env python
import os
import yaml
from collections import OrderedDict
import time
# import shutil
import torch
# import random
# import argparse
# from cStringIO import StringIO

import numpy as np

import torch.nn as nn
import torchvision

from torch.utils import data


from ptsemseg.models import get_model
from ptsemseg.utils import get_logger
# from torch2trt import torch2trt






def softmax(x, axis=None):
    x = x - x.max(axis=axis, keepdims=True)
    y = np.exp(x)
    return y / y.sum(axis=axis, keepdims=True)








def demo(cfg):
     # Setup device

    print(torch.cuda.device_count())
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # Setup Model
    n_classes = cfg["testing"]["n_classes"]
    img_size = (cfg["data"]["img_rows"], cfg["data"]["img_cols"])

    model = get_model(cfg["model"], n_classes)
    total_params = sum(p.numel() for p in model.parameters())
    print( 'Parameters:',total_params )

    if cfg["testing"]["resume"] is not None:
        resume_dir = cfg["testing"]["resume"]
        if os.path.isfile(resume_dir):
            print(
                "Loading model and optimizer from checkpoint '{}'".format(resume_dir)
            )
            checkpoint = torch.load(resume_dir)
            new_state_dict = OrderedDict()
            for k, v in checkpoint["model_state"].items():
                name = k[7:]
                new_state_dict[name] = v
            model.load_state_dict(new_state_dict)
#             model = model.to(device)
            print(
                "Loaded checkpoint '{}' (iter {})".format(
                    resume_dir, checkpoint["epoch"]
                )
            )
        else:
            print("No checkpoint found at '{}'".format(resume_dir))
    model = model.to(device)
    model.eval()
    dummy_input = torch.randn(1, 3, 576, 1024, device='cuda')
    torch.onnx.export(model, dummy_input, "fchardnet.onnx", verbose=True,opset_version=10)
    

    return


    





if __name__ == "__main__":


    # Load basic config from config yaml file --------
    with open("configs/demo.yml") as fp:
        cfg = yaml.load(fp)
    # Load ROS param  --------




  

    demo(cfg)

    


1
('Parameters:', 4118620)
Loading model and optimizer from checkpoint 'weights/0507/hardnet_bdd100k_lane_best_model.pkl'
Loaded checkpoint 'weights/0507/hardnet_bdd100k_lane_best_model.pkl' (iter 30000)


/home/ivslab/.local/lib/python2.7/site-packages/torch/onnx/utils.py:562: UserWarning: ONNX export failed on ATen operator upsample_bilinear2d because torch.onnx.symbolic_opset10.upsample_bilinear2d does not exist
  .format(op_name, opset_version, op_name))


KeyError: u'upsample_bilinear2d'

In [3]:
torch.__version__

'1.2.0+cu92'